In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [156]:
%run -i functions.py

Functions Loaded!


In [161]:
av_raw = pd.read_csv("Availability_for_ML.csv")
fa_nov = pd.read_csv("SCS_Alerts_NOV.csv")
fa_oct = pd.read_csv("SCS_Alerts_OCT.csv")

data = pre_process_av_and_fa_oct_nov(av_raw,fa_oct,fa_nov,remove_same_location_faults = True)

fa = data['faults']
av = data['availability']

fa = floor_time(fa,'Entry time',units='H') 

fa = faults_aggregate_and_pivot(df=fa,
                                time_col='Entry time',
                                fault_level='PLC',
                                agg_col='Duration',
                                agg_type='count',#can use mean/count/sum
                                quadrant=None,# by default all quadrants are kept
                                break_durations=False)

fa  = weight_hours(df=fa, weights = [1,0.5,0.25,0.1])

av = availability_quadrant_mean(df=av,
                                time_col='Datetime',
                                level = 'Module', # by default all data is kept
                                selection = 13) 

df = merge_av_fa(av_df=av,
                 fa_df=fa,
                 min_date=None, 
                 max_date=None) #specify min/max date if you want to filter data by date


#remove columns with only zeros (faults that did not happen in this period of time or quadrant)
df = df.loc[:, (df != 0).any(axis=0)]
df.head()

Quadrants Assigned
Modules Assigned
duplicated location faults removed - max duration kept
Fault and availability data pre-processed
Dates floored
Faults aggregated and pivoted
Previous Hours Weighted
Output will contain data only for Module: 13
Availability data aggregated
Availability and fault datasets merged


,Datetime,Downtime,C05,C06,C07,C08,C09,C10,C11,C12,...,SCSM11,SCSM12,SCSM13,SCSM14,SCSM15,SCSM17,SCSM18,SCSM19,SCSM20,SCSM22
0,2020-10-16 06:00:00,0.409765,11.45,12.75,9.70,28.60,18.90,2.60,16.00,6.80,...,2.80,8.35,3.0,1.10,7.25,1.60,1.85,0.1,1.70,4.10
1,2020-10-16 07:00:00,0.180526,10.90,9.85,10.35,38.65,25.75,3.20,15.95,4.85,...,19.20,10.00,1.5,0.50,7.60,2.75,1.80,0.0,3.70,5.65
2,2020-10-16 08:00:00,0.164189,12.75,16.20,9.70,51.45,23.20,5.45,17.15,13.05,...,10.40,23.50,0.7,0.25,5.75,1.35,3.70,6.0,2.75,5.60
3,2020-10-16 09:00:00,0.180914,22.80,20.60,10.40,38.05,22.95,5.60,15.95,19.30,...,10.00,14.20,1.2,1.10,5.70,3.60,4.75,5.0,3.35,9.70
4,2020-10-16 10:00:00,0.344868,23.10,19.90,17.85,28.75,25.45,5.70,18.40,27.45,...,8.55,20.85,0.5,0.50,2.40,3.70,7.35,2.5,1.55,6.65
